In [1]:
from data.load_idf_10_24 import load_base_model
import pandas as pd

In [31]:
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
data = load_base_model()

Data Frame idf_10_24MODEL récupéré depuis S3


In [4]:
def get_df_i(data : pd.DataFrame) -> list:
    
    df1 = data.copy()
    df2 = data.loc[(data["radié1"] == 0)].copy()
    df3 = data.loc[(data["radié1"] == 0) & (data["radié2"] == 0)].copy()
    df4 = data.loc[(data["radié1"] == 0) & (data["radié2"] == 0) & (data["radié3"] == 0)].copy()
    df5 = data.loc[(data["radié1"] == 0) & (data["radié2"] == 0) & (data["radié3"] == 0) & (data["radié4"] == 0)].copy()

    df1 = df1
    df2 = df2.loc[df2["date_creation"].dt.year < 2023]
    df3 = df3.loc[df3["date_creation"].dt.year < 2022]
    df4 = df4.loc[df4["date_creation"].dt.year < 2021]
    df5 = df5.loc[df5["date_creation"].dt.year < 2020]

    return [df1,df2,df3,df4,df5]

In [5]:
dfs = get_df_i(data)

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, recall_score, precision_score
from sklearn.metrics import confusion_matrix

In [94]:
from xgboost import XGBClassifier

In [459]:
test = dfs[0]

In [460]:
test = test.sort_values(by='date_creation', ascending=True)

In [596]:
X1 = (test[["cp","taille_ville","mean_age","personneMorale","nb_associe","concu/hab","nb_local_concurrents","revDep",
"sumxp","sumxp_rad","code_postal","codeAPE"]])
y1 = test["radié1"]

In [1]:
X1

NameError: name 'X1' is not defined

In [597]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# 1. Identifier colonnes numériques et catégorielles
num_cols = X1.select_dtypes(include=['int64', 'float64']).columns
cat_cols = X1.select_dtypes(include=['object', 'category', 'bool']).columns

# 2. Préprocessing : Standardiser numériques + OneHotEncoder pour catégorielles
preprocess = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols),            # standardisation SEULEMENT des numériques
        ('cat', OneHotEncoder(drop='first'), cat_cols)  # one hot sans standardisation
    ],
    remainder='passthrough'
)


In [ ]:
# 3. Pipeline SMOTE + preprocessing + logistic regression
model = ImbPipeline(steps=[
    ('preprocess', preprocess),
    ('logreg', LogisticRegression(max_iter=1000,class_weight={0:1, 1:80},solver = "liblinear"))
])

# 4. Train/test split
"""
train_obs = int(X1.shape[0]*0.8)
X_train = X1.iloc[0:train_obs,:]
X_test = X1.iloc[train_obs:,:]

y_train = y1.iloc[0:train_obs]
y_test = y1.iloc[train_obs:]
"""
X_train,X_test,y_train,y_test = train_test_split(X1,y1,test_size=0.15,random_state=0)

# 5. Fit
model.fit(X_train, y_train)

,steps,"[('preprocess', ...), ('logreg', ...)]"
,transform_input,None
,memory,None
,verbose,False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'passthrough'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains sparse matrices,these will be stacked as a sparse matrix if the overall density islower than this value. Use ``sparse_threshold=0`` to always returndense. When the transformed output consists of all dense data, thestacked result will be dense, and this keyword will be ignored.",0.3
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None
,"transformer_weights transformer_weights: dict, default=NoneMultiplicative weights for features per transformer. The output of thetransformer is multiplied by these weights. Keys are transformer names,values the weights.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each transformer will beprinted as it is completed.",False
,"verbose_feature_names_out verbose_feature_names_out: bool, str or Callable[[str, str], str], default=True- If True, :meth:`ColumnTransformer.get_feature_names_out` will prefix all feature names with the name of the transformer that generated that feature. It is equivalent to setting `verbose_feature_names_out=""{transformer_name}__{feature_name}""`.- If False, :meth:`ColumnTransformer.get_feature_names_out` will not prefix any feature names and will error if feature names are not unique.- If ``Callable[[str, str], str]``, :meth:`ColumnTransformer.get_feature_names_out` will rename all the features using the name of the transformer. The first argument of the callable is the transformer name and the second argument is the feature name. The returned string will be the new feature name.- If ``str``, it must be a string ready for formatting. Th

In [599]:
y_prob = model.predict_proba(X_test)[:,1]

In [600]:
th = np.linspace(0,1,1000)
f1 = []

for i in th:
    y_pred = [1 if p >= i else 0 for p in y_prob]
    f1.append(f1_score(y_test,y_pred))

max_index = f1.index(np.max(f1))
best_th = th[max_index]

In [601]:
print(best_th)

0.7437437437437437


In [602]:
y_pred = [1 if p >= best_th else 0 for p in y_prob]

In [603]:

# Création de la matrice de confusion
cm = confusion_matrix(y_test, y_pred)

print(cm)
print("auc : ", roc_auc_score(y_test,y_prob))
print("f1 : ", f1_score(y_test,y_pred))
print("accracy : ", accuracy_score(y_test,y_pred))
print("precision : ", precision_score(y_test,y_pred))
print("recall : ", recall_score(y_test,y_pred))

[[16528  4974]
 [  123   350]]
auc :  0.8107050074303526
f1 :  0.12075211316198034
accracy :  0.7680546075085324
precision :  0.06574004507888806
recall :  0.7399577167019028
